<a href="https://colab.research.google.com/github/WalrusEggMan/intro-to-is/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [0]:
import pandas as pd

url = 'https://raw.githubusercontent.com/WalrusEggMan/intro-to-is/master/data.csv'

data = pd.read_csv(url, delimiter=',')
data.head()


# New Section

In [0]:
def fix_seniority_in_company(row):
  if row['seniority in role(years)'] > row['seniority (years) in comapny']:
    return int(row['seniority (years) in comapny'])
  else:
    return int(row['seniority in role(years)'])

def fix_seniority_in_role(row):
  if row['seniority in role(years)'] < row['seniority (years) in comapny']:
    return int(row['seniority (years) in comapny'])
  else:
    return int(row['seniority in role(years)'])

data['fixed_sen_in_role'] = data.apply(fix_seniority_in_role,axis=1)
data['fix_sen_in_comp'] = data.apply(fix_seniority_in_company,axis=1)
data.head(3)

In [0]:
data.describe()

data[data.age - data['fixed_sen_in_role'] <= 16]
#Create new df with clean data
#Delete rows with wrong seniority in role
data.drop('seniority (years) in comapny', axis=1, inplace=True)
data.drop('seniority in role(years)', axis=1, inplace=True)
data_clean = data[data.age - data['fixed_sen_in_role'] > 16]
data_clean = data_clean.reset_index(drop=True)
data_clean.describe()
#All values are not-null values


In [0]:

import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 8, figsize=(16, 4))

i = 0
j = 0

for c in data_clean.columns[1:12]:
    ax[j, i].hist(data_clean[c])
    ax[j, i].set_title(c)
    i = i + 1
    if i == 8:
        j = 1
        i = 0

fig.subplots_adjust(hspace=1, wspace=0.2)
plt.show()


In [0]:
from pandas.plotting import scatter_matrix
# scatter plot matrix
scatter_matrix(data_clean,figsize=(10,10))